# Planted Noisy kXOR: Kikuchi Adjacency List

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `KikuchiNonZeroIndex`
Adjacency list oracle $O_F$ for the Kikuchi matrix.

The oracle $O_F$ (Definition 4.5) takes in $i, k$,
and outputs $i, f(i, k)$ where $f(i, k)$ is
index of the $k$-th non-zero entry in row $i$.

As the Kikuchi matrix is symmetric, we can use the same oracle for both rows and columns.

The Kikuchi matrix is indexed by $S \in {[n] \choose k}$.
For a given row $S$ and column $T$, the entry $\mathcal{K}_{k}_{S, T}$
is potentially non-zero if $S \Delta T = U_j$ for some $j$, which is
equivalent to $T = S \Delta U_j$.
Here, $U_j$ is the $j$-th unique scope in the instance $\mathcal{I}$,
and $\Delta$ is the symmetric difference operator.

See docstring for :class:`KXorInstance` for the overall problem definition.

To find the $k$-th non-zero entry, we use two oracles:
1. $(S, k) \mapsto f(S, k)$, implemented by `ColumnOfKthNonZeroEntry`
2. $(S, f(S, k)) \mapsto k$, implemented by `IndexOfNonZeroColumn`.

Both these above oracles are unitary: they do not have any entangled ancilla/junk registers.


Note on sparsity: This bloq expects the user to provide the sparsity, as it is in general
difficult to compute the precise sparsity of the Kikuchi matrix efficiently. As long as the
provided number is at least the true sparsity, the algorithm will work as expected.
In case the provided sparsity is smaller, it is equivalent to making the remaining entries zero in the final block encoding.

#### Parameters
 - `inst`: the kXOR instance $\mathcal{I}$.
 - `ell`: Kikuchi parameter $\ell$.
 - `s`: sparsity, i.e. max number of non-zero entries in a row/column. 

#### Registers
 - `i`: integer in [2^N]
 - `k`: integer in [2^N] 

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). Theorem 4.17, proof para 4 (top of page 39).


In [ ]:
from qualtran.bloqs.optimization.k_xor_sat import KikuchiNonZeroIndex

### Example Instances

In [ ]:
from qualtran.bloqs.optimization.k_xor_sat.kxor_instance import example_kxor_instance

inst = example_kxor_instance()
ell = 8
s = inst.brute_force_sparsity(ell)

kikuchi_nonzero_index = KikuchiNonZeroIndex(inst, ell, s=s)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([kikuchi_nonzero_index],
           ['`kikuchi_nonzero_index`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
kikuchi_nonzero_index_g, kikuchi_nonzero_index_sigma = kikuchi_nonzero_index.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(kikuchi_nonzero_index_g)
show_counts_sigma(kikuchi_nonzero_index_sigma)